# 第2章　Google Colab, Pandas, Matplotlib, Numpyの基礎

- 保住英希
- 清水秀幸

##### 入力2-1

In [ ]:
# これ以降で使うライブラリの準備
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

##### 入力2-2

In [ ]:
# 下記コードを実行すると，ファイル一覧が取得できる.
# アップロードしたものの他に，デフォルトで入っている
# sample_dataフォルダも表示される.
!ls

##### 入力2-3

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##### 入力2-4

In [ ]:
complete = pd.read_csv('/content/drive/MyDrive/METABRIC_RNA_Mutation.csv')

##### 入力2-5

In [ ]:
!nvidia-smi

##### 入力2-6

In [ ]:
# データの読み込み
# 全部のデータを読み込む(時間がかかることが多い)
# completeとかselectedは好きな単語でよい.df(DataFrameの略)が使われることも多いが，dfでなくてもまったく問題ない
complete = pd.read_csv('METABRIC_RNA_Mutation.csv', low_memory=False)
# 一部のデータのみを抽出する
selected = pd.read_csv('METABRIC_RNA_Mutation.csv',
                       usecols=['age_at_diagnosis', 'cancer_type_detailed', 'cellularity', 
                                      'er_status', 'her2_status', 'mutation_count', 'tumor_stage',
                                      'overall_survival', 'brca1', 'brca2'])

##### 入力2-7

In [ ]:
#データの最初の5行を表示する
selected.head()

##### 入力2-8

In [ ]:
#データの行数、列数を確認する
selected.shape
#データ型を確認する．
selected.dtypes
#カラム名を取得する
selected.columns
#(リストで取得したい場合にはselected.columns.valuesとする)

#以上の3つのコードで得られる内容をまとめて取得する
selected.info()

##### 入力2-9

In [ ]:
# ユニークな値を把握する
factor=selected['cellularity'].unique()
print(factor)
print(selected['cellularity'].value_counts())

##### 入力2-10

In [ ]:
selected.describe()

##### 入力2-11

In [ ]:
plt.rcParams["font.size"] = 12 # フォントサイズを一括して設定する
fig = plt.figure(figsize=(8, 4)) # 8×4サイズの白紙を作る
ax1 = fig.add_subplot(1,2,1) # 白紙であるfigureを1×2に分割し，2つの領域のうち左をax1とする
ax1.set_title(label='Tumor Stage', style = 'italic', weight='bold') # ax1のタイトルのフォント，太さを指定している
ax1.hist(selected['tumor_stage'], bins=5, color='b') # histとはヒストグラムのこと
ax2 = fig.add_subplot(1,2,2,title='Correlation of BRCA1/2') # 1×2の2つの領域のうち右をax2とする
ax2.scatter(selected['brca1'], selected['brca2'], s=2) # scatterとは散布図のこと．s=は散布図の点の大きさ

##### 入力2-12

In [ ]:
# Kaplan Meier曲線を描くためのlifelinesパッケージをインストールする.
!pip install lifelines

##### 入力2-13

In [ ]:
# まず，必要なデータのみを手に入れる．今回は'overall_survial_months', 'her2_status', 'overall_survival'を使う．usecolsは復習
from lifelines import KaplanMeierFitter
breast = pd.read_csv('METABRIC_RNA_Mutation.csv', encoding='cp932', 
                                  usecols=['overall_survival_months', 'her2_status', 'overall_survival'])
breast.info()

##### 入力2-14

In [ ]:
# her2_statusがobjectなので1/0に変更する
# 明示的にどのカテゴリに数値を割り当てるかを辞書形式で指定し，mapメソッドで実行する．2.4節で紹介する
class_her2 = {'Positive':1, 'Negative':0}
breast['her2_status'] = breast['her2_status'].map(class_her2)
# HER2陽性と陰性でデータを分割する
positive = breast.loc[breast['her2_status']==1]
negative = breast.loc[breast['her2_status']==0]
# log-rank検定を行う．lifelinesパッケージの統計手法が入っているフォルダ(.statistics)から log-rankを取り出していると読もう
from lifelines.statistics import logrank_test
logrank = logrank_test(positive['overall_survival_months'], negative['overall_survival_months'], positive['overall_survival'], negative['overall_survival'])
print(logrank)

##### 入力2-15

In [ ]:
from lifelines import KaplanMeierFitter
ax = plt.subplot(111)
# KaplanMeierFitter()をkmp(Kaplan Meier with her2 positiveから適当に名付けた)に割り当てる
kmp = KaplanMeierFitter()
kmp.fit(durations=positive['overall_survival_months'], event_observed=positive['overall_survival'], label='HER2(+)')
# この時点でkmpにはKaplan-Meier統計量の情報が保持されている.ci_showとは信頼区間，show_censors とは打ち切りを表示するか否かという引数である.
ax = kmp.plot(ci_show=True, show_censors=False, ax=ax)
# 今度はHER2 negativeのKaplan-Meier曲線を作る
kmn = KaplanMeierFitter()
kmn.fit(durations=negative['overall_survival_months'], event_observed=negative['overall_survival'], label='HER2(-)')
ax = kmn.plot(ci_show=True, show_censors=False, ax=ax)
ax.set_xlabel('Time(months)', size=10, color='black', style='italic')
ax.set_ylabel('Overall survival', rotation=90, color='blue', fontname='DejaVu Serif')
# p値を図内に表示する.axにannotaionとして加えている.xyには図内の座標を指定する.
ax.annotate('p=0.01', xy=(310, 0.8), xycoords='data', fontsize=10, weight='bold')
plt.show()

##### 入力2-16

In [ ]:
# Seabornをインポートする
import seaborn as sns
# style=に指定する値は図のグリッド(縦横の線)の有無と白黒の色で分類される
sns.set_theme(style='whitegrid')
f, ax = plt.subplots(figsize=(6.5, 6.5))
# despineは不要な軸を除去する
sns.despine(f, left=True, bottom=True)
sns.scatterplot(x='brca1', y='brca2',data=complete)

##### 入力2-17

In [ ]:
# 相関係数を計算したい列(連続変量)を指定する
heatmap = complete[['brca1', 'brca2', 'tp53', 'pten']]
# 相関係数(correlation)を計算する
corr=heatmap.corr()
f, ax = plt.subplots(figsize=(11, 9))
# ヒートマップを描画する
sns.heatmap(corr)

##### 入力2-18

In [ ]:
sns.set_theme(style='ticks')
scatterplot=complete[['cancer_type_detailed', 'brca1', 'fgfr1', 'tp53', 'pten']]
# hue=とすることで，癌の組織型によって色を層別化している
sns.pairplot(scatterplot, hue='cancer_type_detailed')

##### 入力2-19

In [ ]:
from sklearn.decomposition import PCA # scikit-learnは機械学習ライブラリであるが，詳細は後に扱うので本章では割愛する．初めての人はなんとなくでよい．
pca = PCA(n_components=2) # 2次元に圧縮する(n_components=2),PCA(Primary component analysis)インスタンスを作成
X = pca.fit_transform(complete.iloc[:,31:41].values) # brca1~nf1の発現情報をilocで抜き出し，PCAで次元圧縮
embed = pd.DataFrame(X) # 可視化のために結果をデータフレームに変換
embed['label']=complete['overall_survival'] # 可視化用のデータフレームにoverall survivalを加える
import seaborn as sns
sns.set(style='darkgrid')
plt.figure(figsize=(16, 9))
ax = sns.scatterplot(x=embed[0], y=embed[1], hue=embed['label'], data=embed)
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')


##### 入力2-20


In [ ]:
import pandas_profiling as pdp

pdp.ProfileReport(selected)

##### 入力2-21

In [ ]:
!pip install pandas-profiling
!pip install --upgrade pandas_profiling
# 下記については，筆者の場合は必要であったが，大方の場合は不要なはずなので，まずはこれはなしで 次に行ってほしい.
!pip install markupsafe==2.1.1

##### 入力2-22

In [ ]:
selected.isnull().sum()

##### 入力2-23

In [ ]:
deleted=selected.copy()
deleted.dropna(inplace=True)

##### 入力2-24

In [ ]:
# 表に変更を加えるので，遺伝子発現データのみを取得(31列目以降を指定)
genes=complete.iloc[:,31:]
genes.loc[complete['brca1']>0, 'brca1_status']='High_expression'
genes['brca1_status'].value_counts()

##### 入力2-25

In [ ]:
# 表の結合(例)
df1=complete[['patient_id', 'cellularity']]
df2=complete[['patient_id', 'age_at_diagnosis']]
merge=pd.merge(df1, df2, on='patient_id')
comparison=complete[['patient_id', 'cellularity', 'age_at_diagnosis']]
print(merge.equals(comparison))

##### 入力2-26

In [ ]:
# 2.6節の解析ではoverall survivalの説明変数としてcellularity，ERとHER2の発現，BRCA1/2の RNA-seqデータのみ使用するのでそれをdataに格納し，欠損値を削除する
data = pd.read_csv('METABRIC_RNA_Mutation.csv', encoding='cp932',
                              usecols = ['cellularity','er_status', 'her2_status', 'overall_survival', 'brca1', 'brca2'])
data.dropna(inplace=True)

##### 入力2-27

In [ ]:
# カテゴリカル変数に数値を与える. ここではcellularityに1/0の2値を与えた.
data['cellularity'] = data['cellularity'].map({'High': 1, 'Moderate': 1, 'Low': 0})
data['cellularity'].unique()

##### 入力2-28

In [ ]:
# 残ったer_statusなどの項目は2水準しかないのでget_dummiesを使ってみる
data = pd.get_dummies(data, drop_first=True)
data.head()

##### 入力2-29

In [ ]:
# 解析の準備をする.NumPy配列にするために.valuesとしている
# Xに対してはoverall_survival以外を指定してNumPyの配列に変換している
X = data.iloc[:,data.columns!="overall_survival"].values
# Yに対しては直接指定している
Y = data.iloc[:, 1].values

##### 入力2-30

In [ ]:
# 1行目のデータを取得
X[0]

##### 入力2-31

In [ ]:
Y[0]

##### 入力2-32

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# 訓練データとテストデータに分割
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.4, random_state = 0) # 60%のデータを学習データに，40%を検証データにする

lr = LogisticRegression(max_iter=200) # ロジスティック回帰モデルのインスタンスを作成
lr.fit(X_train, Y_train) # ロジスティック回帰モデルの重みを学習

print("coefficient = ", lr.coef_)
print("intercept = ", lr.intercept_)

##### 入力2-33

In [ ]:
X_test[0]

##### 入力2-34

In [ ]:
# X_test[0]の配列を自分で作るなら下記のようにやる
x=np.array([1, -0.408, 1.2402, 0, 0])
# 行列演算をする前には，配列の形状を確認しておく
print(x.shape)
print(lr.coef_.shape)

##### 入力2-35

In [ ]:
# 実際に係数と入力値を求めてロジスティック関数に代入する.
# ロジスティック関数でネイピア数の肩に乗る値を計算するには係数×入力値+切片の値が必要
# dotとは行列積を計算するもの
val=np.dot(lr.coef_, x)
# ロジスティック関数の返り値を計算
probability=1/(1+np.exp(-val-lr.intercept_))
probability

##### 入力2-36

In [ ]:
# テストデータに対する確率を計算
answer=lr.predict_proba(X_test)
# X[0]に該当するデータに対して確率を計算したのでanswer[0]を見る
answer[0]